In [190]:
import csv
import mysql.connector
import json
import logging

from functools import wraps

In [113]:
configFilePath = 'config.json'
productFilePath = None
salesFilepath = None
try:
    with open(configFilePath, 'r') as config_file:
        config = json.load(config_file)
        productFilePath = config.get('product_csv_path')
        salesFilepath = config.get('sales_csv_path')
except Exception as e:
    print(f"Error reading configuration: {e}")

In [139]:
print(productFilePath)
print(salesFilepath)

Product.csv
Sales.csv


In [137]:
def connect_to_database():
    try:
        con = mysql.connector.connect(host='localhost',
            user='root',
            password='root1234',
            port='3306',
            database='testdatabase'
            )
        mycursor = con.cursor()
        return con, mycursor
    except Exception as e:
        print(f"an error of:\n\n{e}\n\noccured while checking if database is empty.")
    finally:
        if con.is_connected():
            print("Closing connector")
            con.close()

In [115]:
def add_csv(file_path, cursor, table_name):
    
    with open(file_path, 'r') as myFile:
        reader = csv.DictReader(myFile)
        
        for row in reader:
            columns = ', '.join(row.keys())
            values = ', '.join(f"'{v}'" for v in row.values())
            query = f"INSERT INTO {table_name} ({columns}) VALUES ({values})"
            
            cursor.execute(query)
         
       

In [117]:
try:
    con = mysql.connector.connect(host='localhost',
        user='root',
        password='root1234',
        port='3306',
        database='testdatabase'
        )
    
    mycursor = con.cursor()
    
    
    #used this right here to creat the 2 tables
    
    mycursor.execute("Create Table Sales (Day DATE, Product varchar(256), Distributor varchar(256), Sales int, salesID int PRIMARY KEY AUTO_INCREMENT )")
    mycursor.execute("Create Table Product (Product varchar(256), Category varchar(256), Brand varchar(256), productID int PRIMARY KEY AUTO_INCREMENT )")
    
    add_csv(productFilePath, mycursor, 'Product')
    add_csv(salesFilepath, mycursor, 'Sales')
    
    
    mycursor.close()
    con.commit()
    
    #openFiles()
except Exception as e:
    print(f"an error of:\n\n{e}\n\noccured while checking if database is empty.")
finally:
    if con.is_connected():
        print("Closing connector")
        con.close()

Closing connector


In [134]:
try:
    con = mysql.connector.connect(host='localhost',
        user='root',
        password='root1234',
        port='3306',
        database='testdatabase'
        )

    mycursor = con.cursor()

    query = """
        CREATE VIEW ProductSalesView AS
        SELECT Sales.Day, Product.Brand, Sales.Sales
        FROM Sales
        INNER JOIN Product ON Sales.Product = Product.Product;
        """
    mycursor.execute(query)
    
    
    mycursor.close()
    con.commit()
    
except Exception as e:
    
    print(f"an error of:\n\n{e}\n\noccured while checking if database is empty.")

finally:
    if con.is_connected():
        print("Closing connector")
        con.close()

Closing connector


In [136]:
try:
    con = mysql.connector.connect(host='localhost',
        user='root',
        password='root1234',
        port='3306',
        database='testdatabase'
        )

    mycursor = con.cursor()
    
    mycursor.execute("SELECT * FROM ProductSalesView")

    # Fetch data from the cursor
    data = mycursor.fetchall()
    
    header = [desc[0] for desc in mycursor.description]
    
    with open('ProductSalesView.csv', 'w', newline='') as csv_file:
            csv_writer = csv.writer(csv_file)
            
            # Write header
            csv_writer.writerow(header)
            
            # Write data rows
            csv_writer.writerows(data)
    
    
except Exception as e:
    
    print(f"an error of:\n\n{e}\n\noccured while checking if database is empty.")

finally:
    if con.is_connected():
        print("Closing connector")
        con.close()

Closing connector


In [272]:
class MySQLLoader:
    
    def __init__(self, config_file_path):
        self.config = self.read_config(config_file_path)        
        if self.config:
            self.mysql_connection = mysql.connector.connect(
                host=self.config['mysql_host'],
                user=self.config['mysql_user'],
                password=self.config['mysql_password'],
                database=self.config['mysql_database']
            )
            self.cursor = self.mysql_connection.cursor()

            # Configure logger
            logging.basicConfig(filename='mysql_loader.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    
    def __del__(self):
        self.close_connection()
        
    def get_mysql_connection(self):
        return self.mysql_connection
    
    def get_cursor(self):
        return self.cursor
    
    def execute_query(self, query):
        self.cursor.execute(query)
        self.mysql_connection.commit()
        
    def close_connection(self):
        if self.mysql_connection.is_connected():
            self.cursor.close()
            self.mysql_connection.close()
            logging.info("MySQL connection closed.")
        
    def read_config(self, file_path):
        try:
            with open(file_path, 'r') as config_file:
                config = json.load(config_file)
            return config
        except Exception as e:
            logging.error(f"Error reading configuration: {e}")
            return None

    def load_csv_to_mysql(self, csv_file_path, table_name):
        try:
            # Read CSV file
            with open(csv_file_path, 'r') as csv_file:
                csv_reader = csv.reader(csv_file)
                header = next(csv_reader)  # Assume the first row is header
                
                # writing a query to check 
                create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ({', '.join(f'{col} VARCHAR(255)' for col in header)});"
                self.cursor.execute(create_table_query)
                logging.info(f"Table '{table_name}' created or already exists.")
           
                # Load data into MySQL
                for row in csv_reader:
                    columns = ', '.join(header)
                    values = ', '.join(f"'{val}'" for val in row)
                    query = f"INSERT INTO {table_name} ({columns}) VALUES ({values})"

                    self.cursor.execute(query)
                logging.info(f"Data from '{csv_file_path}' loaded into '{table_name}'.")
                
                # Commit the changes
                self.mysql_connection.commit()
                logging.info(f"Data from '{csv_file_path}' committed into '{table_name}'.")
                
        except Exception as e:
            logging.error(f"Error loading CSV to MySQL: {e}")


In [273]:
def main():
    config_file_path = 'config.json'
    loader = MySQLLoader(config_file_path)

    if loader.config:
        loader.load_csv_to_mysql(loader.config['product_csv_path'], 'Products')
        loader.load_csv_to_mysql(loader.config['sales_csv_path'], 'Sales')

        
    query = """
        CREATE OR REPLACE VIEW ProductsSalesView AS
        SELECT Sales.Day, Products.Brand, Sales.Sales
        FROM Sales
        INNER JOIN Products ON Sales.Product = Products.Product;
        """
    loader.execute_query(query)
    
    mycursor = loader.get_cursor()
    mycursor.execute("SELECT * FROM ProductsSalesView")

    # Fetch data from the cursor
    data = mycursor.fetchall()
    
    header = [desc[0] for desc in mycursor.description]
    
    with open('ProductSalesView.csv', 'w', newline='') as csv_file:
            csv_writer = csv.writer(csv_file)
            
            # Write header
            csv_writer.writerow(header)
            
            # Write data rows
            csv_writer.writerows(data)
    
    loader.close_connection()

In [274]:
if __name__ == '__main__':
    main()

In [227]:
with open('Products.csv', 'r') as csv_file:
    
    table_name = 'Products'
    
    csv_reader = csv.reader(csv_file)
    header = next(csv_reader)  # Assume the first row is header

    print(header)
    columns = ', '.join(header)
    
    
    print(columns)
    for row in csv_reader:
        values = ', '.join(f"'{v}'" for v in row)
        query = f"INSERT INTO {table_name} ({columns}) VALUES ({values})"
        print(query)

['Product', 'Category', 'Brand']
Product, Category, Brand
INSERT INTO Products.csv (Product, Category, Brand) VALUES ('Product_A', 'Electronics', 'Brand_X')
INSERT INTO Products.csv (Product, Category, Brand) VALUES ('Product_B', 'Appliances', 'Brand_Y')
INSERT INTO Products.csv (Product, Category, Brand) VALUES ('Product_C', 'Clothing', 'Brand_Z')


In [176]:
class DataLoader:
    def __init__(self, configFilePath, jasonText, tableName):
        
        self.table_Name = tableName
        
        self.my_connection, self.my_cursor = self.__connect_to_database()
        
        self.open_Jason_ConfigFile(configFilePath, jasonText)
        
    def __del__(self):
        self.my_cursor.close()
        self.my_connection.close()
        
    def __connect_to_database(self):
        try:
            con = mysql.connector.connect(host='localhost',
                user='root',
                password='root1234',
                port='3306',
                database='testdatabase'
                )
            mycursor = con.cursor()
            return con, mycursor
        except Exception as e:
            #Needs to be logged
            print(f"an error of:\n\n{e}\n\noccured while checking if database is empty.")
            return None , None
        
    def open_Jason_ConfigFile(self, configFilePath, jasonText):
        print(configFilePath)
        print(jasonText)
        try:
            with open(configFilePath, 'r') as config_file:
                config = json.load(config_file)
                myFilepath = config.get(jasonText)
                
        except Exception as e:
            #needs to be logged
            print(f"Error reading configuration: {e}")